#### Домашнее задание №6

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("../../data/pima-indians-diabetes.data.csv", 
                    names=['pregnancies_count', 'glucose', 'arterial_pressure',
                           'th_skinfold_thickness', '2h_insulin', 'body_mass_index',
                           'diabetes_pedigree', 'age', 'diabetes'
                          ])

###### Набор данных диабета индейцев Пима
Набор данных о диабете индейцев пима включает прогнозирование возникновения диабета в течение 5 лет у индейцев пима с учетом медицинских данных.

Это бинарная (2-классная) задача классификации. Количество наблюдений для каждого класса не сбалансировано. Есть 768 наблюдений с 8 входными переменными и 1 выходной переменной. Считается, что пропущенные значения кодируются нулевыми значениями. Имена переменных следующие:

0. Количество раз беременных.
1. Концентрация глюкозы в плазме через 2 часа при оральном тесте на толерантность к глюкозе.
2. Диастолическое артериальное давление (мм рт. Ст.).
3. Толщина трехглавой кожной складки (мм).
4. 2-часовой сывороточный инсулин (мю Ед / мл).
5. Индекс массы тела (вес в кг / (рост в м) ^ 2).
6. Родословная функция диабета.
7. Возраст (годы).
8. Переменная класса (0 или 1).

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   pregnancies_count      768 non-null    int64  
 1   glucose                768 non-null    int64  
 2   arterial_pressure      768 non-null    int64  
 3   th_skinfold_thickness  768 non-null    int64  
 4   2h_insulin             768 non-null    int64  
 5   body_mass_index        768 non-null    float64
 6   diabetes_pedigree      768 non-null    float64
 7   age                    768 non-null    int64  
 8   diabetes               768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [3]:
data.head()

,pregnancies_count,glucose,arterial_pressure,th_skinfold_thickness,2h_insulin,body_mass_index,diabetes_pedigree,age,diabetes
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
target_name = 'diabetes'

In [5]:
data[target_name].value_counts()

0    500
1    268
Name: diabetes, dtype: int64

In [6]:
train = data.drop(target_name, 1)
target = data[target_name]

In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    train, target, test_size=0.2, random_state=7)

In [8]:
import catboost as cb

model = cb.CatBoostClassifier()

model.fit(x_train, y_train, silent=True)
y_predict = model.predict(x_test)

Проверяем качество

In [9]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 

    
evaluate_results(y_test, y_predict)

Classification results:
f1: 70.59%
roc: 76.94%
recall: 63.16%
precision: 80.00%


### Теперь очередь за PU learning

Представим, что нам неизвестны негативы и часть позитивов

In [10]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 67/268 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [11]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    701
 1     67
Name: class_test, dtype: int64


* We now have just 153 positive samples labeled as 1 in the 'class_test' col while the rest is unlabeled as -1.

* Recall that col 4 still holds the actual label

In [12]:
mod_data.head(10)

,pregnancies_count,glucose,arterial_pressure,th_skinfold_thickness,2h_insulin,body_mass_index,diabetes_pedigree,age,diabetes,class_test
0,6,148,72,35,0,33.6,0.627,50,1,1
1,1,85,66,29,0,26.6,0.351,31,0,-1
2,8,183,64,0,0,23.3,0.672,32,1,-1
3,1,89,66,23,94,28.1,0.167,21,0,-1
4,0,137,40,35,168,43.1,2.288,33,1,1
5,5,116,74,0,0,25.6,0.201,30,0,-1
6,3,78,50,32,88,31.0,0.248,26,1,-1
7,10,115,0,0,0,35.3,0.134,29,0,-1
8,2,197,70,45,543,30.5,0.158,53,1,-1
9,8,125,96,0,0,0.0,0.232,54,1,-1


Remember that this data frame (x_data) includes the former target variable that we keep here just to compare the results

[:-2] is the original class label for positive and negative data [:-1] is the new class for positive and unlabeled data

In [13]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

### 1. random negative sampling

In [14]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(67, 10) (67, 10)


In [16]:
model = cb.CatBoostClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values, silent=True)
y_predict = model.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 59.03%
roc: 71.42%
recall: 82.61%
precision: 45.92%


### Домашнее задание

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
3. сделать feature engineering
4. обучить любой классификатор (какой вам нравится)
5. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
6. применить random negative sampling для построения классификатора в новых условиях
7. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
8. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

<b>Бонусный вопрос:</b>

Как вы думаете, какой из методов на практике является более предпочтительным: random negative sampling или 2-step approach?

Ваш ответ здесь: